# Random Forest: Classification by Automation Condition (Random Sampling)
- Prediction of automation usage using all features and the top 20
- We train and test by automation condition
    - Grouping the data from all participants in a condition

## Necessary Libraries

In [2]:
# Import libraries
import pandas as pd
import numpy as np
import glob as glob
import os as os
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from joblib import Parallel, delayed
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import *

## Reading and processing the data

### Choose if running on Agave

In [3]:
AGAVE = False


### Choose automation condition

In [4]:
choose_condition  = 0
conditions = ['SH','SL','FH','FL','ALL']
cond = conditions[choose_condition]

### Path to all data files

In [5]:
# Files
if(AGAVE==True):
    files_path = '../../../NewFeatures/' + cond             # Agave
else:
    files_path = '../../../features_data_risk/' + cond      # Local

all_files = glob.glob(os.path.join(files_path, "*.csv"))

### Features that offer extra information

In [6]:
# Features that offer extra information (they're basically the same as boolAuto)
features_extra = ['boolHand','boolTake','brakeOshp','sumAuto','sumHand','sumTake','sumTogg']

### Features that are related to the operator's actions

In [7]:
# Actions
features_internal = ['accAngOshpX','accAngOshpY','accAngOshpZ','accLinOshpX',
					'accLinOshpY','accLinOshpZ','boolButnA','boolButnB',
					'boolViolButn','boolViolLane','boolViolLead','boolViolPeds',
					'boolViolRang','boolViolTraf','boolViolVehs',
					'oriOshpX','oriOshpY','oriOshpZ','psnOshpLane','psnOshpLaneAbs',
					'psnOshpLaneLft','psnOshpX','psnOshpY','psnOshpZ',
					'steerOshp','sumViolButn','sumViolLane','sumViolLead','sumViolPeds',
					'sumViolRang','sumViolTraf','sumViolVehs','throtOshp','timeReact',
					'velAngOshpX','velAngOshpY','velAngOshpZ','velLinOshp','velLinOshpLane',
					'velLinOshpLaneAbs','velLinOshpLaneLft','velLinOshpRang','velLinOshpX',
					'velLinOshpY','velLinOshpZ']

### Read the data files

In [8]:
# Pre-process data
# Assign features from current time-step to future use of automation,
# The idea is to predict future use of automation based on current behavior
delay = 1/60            # How much time in advance we want to predict automation usage (in seconds)
h = (int)(delay*60 - 1) # Turn the time into index (Considering a sampling rate of 60 Hz)
data_list = []
for file in all_files:
    data_participant = pd.read_csv(file)
    data_participant.insert(0,"time",[(i*1/60) for i in range(0,data_participant.shape[0])],True)
    data_auto = data_participant['boolAuto']
    data_auto = np.array(data_auto.iloc[h:])
    data_participant = data_participant.drop('boolAuto', axis=1)
    data_participant = data_participant.head(data_participant.shape[0]-h)
    data_participant.insert(data_participant.shape[1],"boolAuto",data_auto,True)
    data_list.append(data_participant)

In [9]:
# Put the data from all participants together
df_data = pd.concat(data_list, ignore_index=True) # Read the data from all participants

### Delete features that offer extra information

In [10]:
# Delete the features that offer extra information from the dataset
# The idea is to determine which features are more related to automation usage
df_data = df_data.drop(features_extra,axis = 1)
df_data = df_data.drop('time',axis=1)               # Drop the inserted time too
# df_data = df_data.drop(features_internal,axis = 1)

In [11]:
# Take just a sample of the data for speed
# Change this when running on Agave
SEED = 10
if(AGAVE==True):
    sample_data = df_data.sample(frac=1, random_state=SEED);
else:
    sample_data = df_data.sample(frac=0.1, random_state=SEED);

In [12]:
X_data = sample_data.drop('boolAuto',axis = 1)
y_data = sample_data['boolAuto']

### Results of feature selection

In [13]:
SH_feats = ['psnOshpRang'    ,
'score'          ,
'velLinOshpX'    ,
'velLinOshp'     ,
'rrisk'          ,
'sumViolLane'    ,
'velLinLead'     ,
'throtOshp'      ,
'sumViolRang'    ,
'odomRoad'       ,
'velLinOshpRang' ,
'sumViolPeds'    ,
'ttcLead'        ,
'sumViolButn'    ,
'accLinOshpX'    ,
'psnTrafPrxY'    ,
'psnTrafPrxX'    ,
'boolStatRang'   ,
'oriTrafPrxZ'    ,
'oriOshpY'       ]

SL_feats = ['psnOshpRang'    ,
'rrisk'          ,
'sumViolAwrd'    ,
'velLinOshp'     ,
'throtOshp'      ,
'velLinOshpX'    ,
'sumViolRang'    ,
'score'          ,
'accLinOshpX'    ,
'sumViolButn'    ,
'oriOshpY'       ,
'velLinLead'     ,
'odomRoad'       ,
'sumViolLane'    ,
'velLinOshpRang' ,
'psnTrafPrxX'    ,
'psnTrafPrxY'    ,
'accAngOshpY'    ,
'oriLeadZ'       ,
'velAngOshpY'    ]

FH_feats = ['velLinOshpX'    ,
'psnOshpRang'    ,
'score'          ,
'rrisk'          ,
'velLinOshp'     ,
'throtOshp'      ,
'sumViolRang'    ,
'velLinLead'     ,
'odomRoad'       ,
'sumViolButn'    ,
'sumViolLane'    ,
'accLinOshpX'    ,
'velLinOshpRang' ,
'psnOshpLaneLft' ,
'oriTrafPrxZ'    ,
'velAngLeadZAbs' ,
'psnTrafPrxX'    ,
'oriOshpY'       ,
'psnTrafPrxY'    ,
'oriOshpZ'       ]

FL_feats = ['psnOshpRang' ,
'sumViolAwrd' ,
'velLinOshpX' ,
'rrisk'       ,
'throtOshp'   ,
'velLinOshp'  ,
'sumViolLane' ,
'score'       ,
'accLinOshpX' ,
'odomRoad'    ,
'sumViolRang' ,
'oriTrafPrxZ' ,
'sumViolButn' ,
'psnTrafPrxX' ,
'velLinLead'  ,
'sumViolPeds' ,
'psnLeadY'    ,
'timeReact'   ,
'psnTrafPrxY' ,
'psnRoadY'    ]

top_features_all = [SH_feats,SL_feats,FH_feats,FL_feats]
top_feat_cond = top_features_all[choose_condition]


In [14]:
# Update the dataset to consider only the top num_feats features
X_data_cut = X_data[top_feat_cond]
# Scale the data (0,1)
X_data_cut = MinMaxScaler().fit_transform(X_data_cut)
y_data = np.array(y_data.to_list())
# X_data = X_data[top_feat_cond]

## Split the data into training and testing

In [15]:
# Split the data into training and testing
frac = 0.8 # Choose fraction of data to use for training
n_samples = len(X_data_cut)
idx_split = (int)(np.round(frac*n_samples))

In [16]:
# Split data into features and target
train_labels = np.array(y_data[0:idx_split])
test_labels = np.array(y_data[idx_split:n_samples]) 
train_features = np.array(X_data_cut[0:idx_split,:])
test_features = np.array(X_data_cut[idx_split:n_samples,:])

In [17]:
# Check sizes
print('Training Data Shape:', train_features.shape)
print('Testing Data Shape:', test_features.shape)

Training Data Shape: (55765, 20)
Testing Data Shape: (13941, 20)


## Evaluate the classification models

In [18]:
# Save the performance results
data_scores = {'Train_Acc':[],'Train_BalAcc':[],'Train_Prec':[],'Train_Rec':[],'Train_Spec':[],
                'Train_AUC':[],'Test_Acc':[],'Test_BalAcc':[],'Test_Prec':[],'Test_Rec':[],
                'Test_Spec':[],'Test_AUC':[]}
perf_scores = pd.DataFrame(data=data_scores)
# Save the ROC curve data
data_roc = pd.DataFrame({'mean_fpr':[], 'tpr_1':[], 'tpr_2':[], 'tpr_3':[], 'tpr_4':[], 
                        'tpr_5':[]}) # 5 KFold
# Save the best set of parameter for the model
best_parameters = pd.DataFrame({'n_estimators':[], 'max_depth':[], 'max_features':[]})

### Random Forest

#### Random Grid Search

In [25]:
# Get the best set of parameters, using Grid Search
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 1000, num = 10)]
# Maximum number of levels in tree
max_num_depth = 6000
if(AGAVE==True):
    max_num_depth = max_num_depth*10
max_depth = [int(x) for x in np.linspace(100, max_num_depth, num = 15)]
max_depth.append(None)
# Maximum number of features
max_features = [4,5,8,10,15]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_depth': max_depth,
               'max_features': max_features}

In [26]:
max_depth

[100,
 521,
 942,
 1364,
 1785,
 2207,
 2628,
 3050,
 3471,
 3892,
 4314,
 4735,
 5157,
 5578,
 6000,
 None]

In [135]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf_gs = RandomForestClassifier(n_jobs=-1)
# Random search of parameters, using 5 fold cross validation, 
# Search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf_gs, param_distributions = random_grid, 
                                n_iter = 100, cv = 5, verbose=2, n_jobs = -1)
# Fit the random search model
rf_random.fit(train_features, train_labels)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(), n_jobs=6,
                   param_distributions={'max_depth': [10, 1204, 2398, 3592,
                                                      4787, 5981, 7175, 8369,
                                                      9564, 10758, 11952, 13146,
                                                      14341, 15535, 16729,
                                                      None],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [10, 120, 230, 340, 450,
                                                         560, 670, 780, 890,
                                                         1000]},
                   random_state=10, verbose=2)

#### Cross Validation (5-fold)

In [206]:
# Cross Validation when the best parameters have been selected
best_n_estim = rf_random.best_params_['n_estimators']
best_max_depth = rf_random.best_params_['max_depth']
best_max_features = rf_random.best_params_['max_features']

cv = StratifiedKFold(n_splits=5)
classifier = RandomForestClassifier(n_estimators=best_n_estim, max_depth=best_max_depth,
                            max_features=best_max_features, n_jobs=-1)

tprs = []; aucs = []; acc = []; acc_bal = []; prec = []; rec = []; spec = []; AUC_v = []
acc_tr = []; acc_bal_tr = []; prec_tr = []; rec_tr = []; spec_tr = []; AUC_v_tr = []
mean_fpr = np.linspace(0, 1, 100)

for i, (train, test) in enumerate(cv.split(X_data_cut, y_data)):
    # Classifier training and testing
    y_train = y_data[train]
    classifier.fit(X_data_cut[train], y_train)
    y_test = y_data[test]
    y_pred = classifier.predict(X_data_cut[test])
    y_pred_tr = classifier.predict(X_data_cut[train])

    # ROC curve metrics
    fpr, tpr, _ = roc_curve(y_data[test], y_pred, pos_label=1)
    interp_tpr = np.interp(mean_fpr, fpr, tpr)
    interp_tpr[0] = 0.0
    tprs.append(interp_tpr)

    # Other performance metrics
    # Testing
    acc.append(accuracy_score(y_test,y_pred))
    acc_bal.append(balanced_accuracy_score(y_test,y_pred))
    prec.append(precision_score(y_test,y_pred))
    rec.append(recall_score(y_test,y_pred, pos_label=1))
    spec.append(recall_score(y_test,y_pred, pos_label=0))
    AUC_v.append(roc_auc_score(y_test,y_pred))
    # Training
    acc_tr.append(accuracy_score(y_train,y_pred_tr))
    acc_bal_tr.append(balanced_accuracy_score(y_train,y_pred_tr))
    prec_tr.append(precision_score(y_train,y_pred_tr))
    rec_tr.append(recall_score(y_train,y_pred_tr, pos_label=1))
    spec_tr.append(recall_score(y_train,y_pred_tr, pos_label=0))
    AUC_v_tr.append(roc_auc_score(y_train,y_pred_tr))

#### Save the results

In [207]:
# Fill the dataframe with all the results
perf_scores.Train_Acc = acc_tr
perf_scores.Train_BalAcc = acc_bal_tr
perf_scores.Train_Prec = prec_tr
perf_scores.Train_Rec = rec_tr
perf_scores.Train_Spec = spec_tr
perf_scores.Train_AUC = AUC_v_tr
perf_scores.Test_Acc = acc
perf_scores.Test_BalAcc	 = acc_bal
perf_scores.Test_Prec = prec
perf_scores.Test_Rec = rec
perf_scores.Test_Spec = spec
perf_scores.Test_AUC = AUC_v
# Best parameters
best_parameters.n_estimators = [best_n_estim]
best_parameters.max_depth = [best_max_depth]
best_parameters.max_features = [best_max_features]
# Data ROC curves
data_roc.mean_fpr = mean_fpr
for i in range(1,6):
    data_roc.iloc[:,i] = tprs[i-1]

In [212]:
# Save the results to .csv files
folder = 'Resuls_CM'
perf_scores.to_csv(folder + '/RF_CRS_performance_' + cond + '.csv', index=False)
data_roc.to_csv(folder + '/RF_CRS_ROC_' + cond + '.csv', index=False)
best_parameters.to_csv(folder + '/RF_CRS_BestParm_' + cond + '.csv', index=False)